In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score,mean_squared_error

c:\Users\David\Documents\Coding\League Stats\venv\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [9]:
import os
import glob
filenames = [i for i in glob.glob(os.path.join(os.getcwd() + '\\dataset', '*.csv'))]
training_dataset = pd.concat([pd.read_csv(f) for f in filenames])
training_dataset.drop(training_dataset.filter(regex='^armor|^attack|championTransform|consumablesPurchased|damageSelfMitigated|detectorWardsPlaced|^firstBlood|gameEnded|^hp|^item|largestCriticalStrike|movespeed|^mp|^nexus|^objective|participantId|pentaKills|physicalDamageTaken|profileIcon|^spell|^summoner[1-2]|summonerLevel|summonerId|^team|^time|totalDamageShielded|totalHealsOnTeammates|totalMinionsKilled|totalTimeCC|totalUnitsHealed|^trueDamage|visionWards|^wards').columns, axis=1, inplace=True)
training_dataset.to_csv('dataset/training_dataset_021922_1914.csv', index = False)

In [38]:
stubnames = list(set([x[0] for x in training_dataset.columns.str.split('_')]))

In [39]:
training_dataset_long = pd.wide_to_long(training_dataset.reset_index(), stubnames, i = 'index', j = 'num', sep = '_')

In [45]:
len(training_dataset.columns)

311

In [43]:
training_dataset_long.columns.sort_values()[:25]

Index(['assists', 'baronKills', 'bountyLevel', 'champExperience', 'champLevel',
       'crit', 'critperlevel', 'damageDealtToBuildings',
       'damageDealtToObjectives', 'damageDealtToTurrets', 'deaths',
       'doubleKills', 'dragonKills', 'firstTowerAssist', 'firstTowerKill',
       'goldEarned', 'goldSpent', 'inhibitorKills', 'inhibitorTakedowns',
       'inhibitorsLost', 'killingSprees', 'kills', 'largestKillingSpree',
       'largestMultiKill', 'longestTimeSpentLiving'],
      dtype='object')

In [42]:
training_dataset_long.drop(training_dataset_long.filter(regex='^armor|^attack|championTransform|consumablesPurchased|damageSelfMitigated|detectorWardsPlaced|^firstBlood|gameEnded|^hp|^item|largestCriticalStrike|movespeed|^mp|^nexus|^objective|participantId|pentaKills|physicalDamageTaken|profileIcon|^spell|^summoner[1-2]|summonerLevel|summonerId|^team|^time|totalDamageShielded|totalHealsOnTeammates|totalMinionsKilled|totalTimeCC|totalUnitsHealed|^trueDamage|visionWards|^wards').columns, axis=1, inplace=True)

In [5]:
corr = training_dataset_long.corrwith(training_dataset_long['win_'])
corr_exclude = corr[abs(corr) < 0.05]

In [11]:
corr_exclude.sort_index()[50:75]

totalMinionsKilled            0.028849
totalTimeCCDealt              0.042780
totalUnitsHealed              0.031599
trueDamageDealt               0.028987
trueDamageDealtToChampions    0.048810
trueDamageTaken              -0.027781
visionWardsBoughtInGame       0.038870
wardsKilled                   0.046672
wardsPlaced                   0.027788
dtype: float64

In [16]:
training_dataset_long.filter(like='summoner')


,,summonerLevel,summoner2Id,summoner2Casts,summoner1Id,summoner1Casts
index,num,,,,,
0,0,635.0,14.0,6.0,12.0,4.0
1,0,140.0,4.0,5.0,14.0,7.0
2,0,142.0,12.0,2.0,4.0,3.0
3,0,762.0,12.0,4.0,4.0,4.0
4,0,513.0,3.0,3.0,4.0,2.0
...,...,...,...,...,...,...
1929,4,461.0,4.0,8.0,14.0,4.0
1930,4,462.0,3.0,6.0,14.0,7.0
1931,4,771.0,4.0,3.0,7.0,6.0
